# Instituto Tecnologico de Costa Rica
# Ingenieria en Computacion
# Inteligencia artificial 
--- 
## Tarea Programada 3 
## Profesora : Maria Auxialiadora Mora
---
## Estudiantes: 
## Kenneth Corrales Lizano 2017151342
## Fabian Vives Castro 2018319257
---

In [89]:
#Bibliotecas
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import re
import string

#Bibliotecas Ejemplo 2
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import spacy
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt


&nbsp;

# Ejercicio 1

### Describa el problema y el objetivo del ejercicio

### Describa los datos utilizados en el ejercicio

### Cargue, normalice y explore los datos


&nbsp;
# Ejercicio 2

### 1. 
Describa el problema y el objetivo del ejercicio.

A la hora de comprar un producto, ir a algun lugar, realizar alguna actividad..., basamos ya sea consiente o inconscientemente nuestras decisiones en base a opiniones externas, es por esto que muchas companias toman en especial cuidado el como lo ven sus consumidores ya que estos son los que posteriormente agregaran nueva clientela. Es por esto que es de suma importancia tener la calificacion y la informacion de los consumidores, para tomar decisiones al respecto de que esta bien, que es mejor cambiar asi como ideas de donde innovar. Ahora bien puesto que leer miles o millones de criticas redactadas en texto no es una tarea tan rapida, acudimos  a la tecnologia, en este caso en especifico a un algoritmo de procesamiento de lenguaje natural llamado analisis de sentimiento, en el que a groso modo, pretende clasificar cada critica  en clases definidas por el investigador, a fin de en una manera rapida, conocer los promedios de que tan satisfecho/insatisfecho se encuentran los usuarios de la plataforma tripadvisor en este caso.

### 2. Describa los datos utilizados en el ejercicio.

Como se esbozo anteriormente, se van a analizar una recopilacion de las criticas a los hoteles que se encuentran en la plataforma tripadvisor, este dataset consta de 2 columnas unicamente, la primera se refiere al detalle de la critica, es decir una breve descripcion de la misma, la segunda es un valor de estrellas del 1 al 5. La cantidad de filas es ligeramente superior a los 20 mil records.



### 3. Cargue y prepare los datos para ser introducidos a la red LSTM.

In [71]:
DEBUG = 0
path= "tripadvisor_hotel_reviews.csv"
raw_data =pd.read_csv(path)

# podemos ver las dimensiones de la informacion
print(f"Tenemos {raw_data.shape[0]} filas  y {raw_data.shape[1]} columnas: el review y el rating")
# una visualizacion de las primeras filas de la data
raw_data.head(10)

Tenemos 20491 filas  y 2 columnas: el review y el rating


,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
5,love monaco staff husband stayed hotel crazy w...,5
6,"cozy stay rainy city, husband spent 7 nights m...",5
7,"excellent staff, housekeeping quality hotel ch...",4
8,"hotel stayed hotel monaco cruise, rooms genero...",5
9,excellent stayed hotel monaco past w/e delight...,5


In [72]:
#Checamos si hay Nulos y los remplazamos con 
raw_data['Review'] = raw_data['Review'].fillna('')
raw_data['Rating'] = raw_data['Rating'].fillna('')

In [73]:

# Segun un chequeo general del documento parece que la data no tiene espacios nulos o vacios sin embargo, 
# puesto que son 20mil lineas lo mejor es que el sistema lo haga por nosotros
print("Cantidad de registros antes de la limpieza", raw_data.shape)
# iniciamos la variable de la data 
parsed_data = raw_data
# Creamos una variable con la informacion una vez parseada 
parsed_data['Review_Length'] = parsed_data['Review'].apply(lambda x: len(x.split()))
parsed_data = parsed_data[parsed_data['Review_Length'] != 0]
parsed_data = parsed_data[pd.notnull(parsed_data['Rating'])]
parsed_data = parsed_data[parsed_data['Rating'] != 0]
print("Cantidad de registros después de la limpieza", raw_data.shape)

Cantidad de registros antes de la limpieza (20491, 2)
Cantidad de registros después de la limpieza (20491, 3)


In [74]:
# Se debe crear el vocabulario pues es de lo que se va a alimentar principalmente la RNN
#puesto que 
tok = spacy.blank("en")
from spacy.lang.en import English

nlp = English()
from spacy.lang.en.stop_words import STOP_WORDS
# Tokenize se encarga de convertir la linea de la critica en un set de tokens es decir separa las oraciones en palabras
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    if(DEBUG):
        print(nopunct)
    # Eliminamos los stopwords, para mejorar el rendimiento, ya que no nos aportan informacion valiosa
    lexema = nlp.vocab[nopunct]
    if lexema.is_stop == False:  
        return [token.text for token in tok.tokenizer(nopunct)]

In [75]:
#Utilizando la funcion anterior creamos la cuenta de las palabras iterando sobre la lista de criticas
counts = Counter()
for index, row in parsed_data.iterrows():
    counts.update(tokenize(row['Review']))

In [76]:
# # borramos las palabras poco utilizadas con el fin de reducir el vocabulario y mejorar las clases
# print("num_words before:",len(counts.keys()))
# for word in list(counts):
#     if counts[word] < 2:
#         del counts[word]
# print("num_words after:",len(counts.keys()))
# print(counts)

In [77]:
# Creamos el vocabulario, si una palabra no existe se anade al UNK
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [78]:
# recibe el texto el vocabulario y un tamanio
# el valor N se setea en 100 debido a un aproximado al promedio de palabras por critica
# esta funcion se encarga de convertir cada palabra de la critica a su codificacion en el vocabulario, si no encuentra la palabra
# la cambia por el valor que le corresponde a "UNK"
def encode_sentence(text, vocab2index, N=100):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    
    # El get en diccionario permite definir un valor si un item no existe ("UNK").  
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    
    # Largo máximo del resultado.
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

parsed_data['encoded'] = parsed_data['Review'].apply(lambda x: np.array(encode_sentence(x,vocab2index ), dtype=object))
print(parsed_data.head())

                                              Review  Rating  Review_Length  \
0  nice hotel expensive parking got good deal sta...       4             87   
1  ok nothing special charge diamond member hilto...       2            250   
2  nice rooms not 4* experience hotel monaco seat...       3            217   
3  unique, great stay, wonderful time hotel monac...       5             89   
4  great stay great stay, went seahawk game aweso...       5            191   

                                             encoded  
0  [[2, 3, 4, 5, 6, 7, 8, 9, 3, 10, 11, 12, 13, 1...  
1  [[79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 78, ...  
2  [[2, 235, 39, 77, 74, 3, 236, 90, 7, 3, 237, 2...  
3  [[350, 11, 69, 9, 11, 351, 352, 3, 236, 11, 68...  
4  [[69, 9, 69, 9, 11, 398, 399, 400, 401, 11, 40...  


### 4. Estadisticas


In [79]:
parsed_data['Review_Length'] = parsed_data['Review'].apply(lambda x: len(x.split()))
parsed_data.head(10)

,Review,Rating,Review_Length,encoded
0,nice hotel expensive parking got good deal sta...,4,87,"[[2, 3, 4, 5, 6, 7, 8, 9, 3, 10, 11, 12, 13, 1..."
1,ok nothing special charge diamond member hilto...,2,250,"[[79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 78, ..."
2,nice rooms not 4* experience hotel monaco seat...,3,217,"[[2, 235, 39, 77, 74, 3, 236, 90, 7, 3, 237, 2..."
3,"unique, great stay, wonderful time hotel monac...",5,89,"[[350, 11, 69, 9, 11, 351, 352, 3, 236, 11, 68..."
4,"great stay great stay, went seahawk game aweso...",5,191,"[[69, 9, 69, 9, 11, 398, 399, 400, 401, 11, 40..."
5,love monaco staff husband stayed hotel crazy w...,5,134,"[[495, 236, 154, 496, 497, 3, 498, 260, 499, 5..."
6,"cozy stay rainy city, husband spent 7 nights m...",5,101,"[[580, 9, 581, 427, 11, 496, 268, 78, 582, 236..."
7,"excellent staff, housekeeping quality hotel ch...",4,85,"[[353, 154, 11, 207, 624, 3, 625, 154, 191, 39..."
8,"hotel stayed hotel monaco cruise, rooms genero...",5,59,"[[3, 497, 3, 236, 655, 11, 235, 603, 517, 656,..."
9,excellent stayed hotel monaco past w/e delight...,5,35,"[[353, 497, 3, 236, 681, 267, 682, 683, 11, 53..."


In [80]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
parsed_data.describe()

,Rating,Review_Length
count,20491.00,20491.00
mean,3.95,104.38
std,1.23,100.66
min,1.00,7.00
25%,3.00,48.00
50%,4.00,77.00
75%,5.00,124.00
max,5.00,1931.00


In [81]:
# con esto podemos ordenar/ limpiar los datos para que sean mas facil trabajar 
parsed_data['Rating'].replace({1: 0, 2: 1, 3: 2, 4: 3, 5: 4}, inplace=True)
parsed_data.head(10)

,Review,Rating,Review_Length,encoded
0,nice hotel expensive parking got good deal sta...,3,87,"[[2, 3, 4, 5, 6, 7, 8, 9, 3, 10, 11, 12, 13, 1..."
1,ok nothing special charge diamond member hilto...,1,250,"[[79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 78, ..."
2,nice rooms not 4* experience hotel monaco seat...,2,217,"[[2, 235, 39, 77, 74, 3, 236, 90, 7, 3, 237, 2..."
3,"unique, great stay, wonderful time hotel monac...",4,89,"[[350, 11, 69, 9, 11, 351, 352, 3, 236, 11, 68..."
4,"great stay great stay, went seahawk game aweso...",4,191,"[[69, 9, 69, 9, 11, 398, 399, 400, 401, 11, 40..."
5,love monaco staff husband stayed hotel crazy w...,4,134,"[[495, 236, 154, 496, 497, 3, 498, 260, 499, 5..."
6,"cozy stay rainy city, husband spent 7 nights m...",4,101,"[[580, 9, 581, 427, 11, 496, 268, 78, 582, 236..."
7,"excellent staff, housekeeping quality hotel ch...",3,85,"[[353, 154, 11, 207, 624, 3, 625, 154, 191, 39..."
8,"hotel stayed hotel monaco cruise, rooms genero...",4,59,"[[3, 497, 3, 236, 655, 11, 235, 603, 517, 656,..."
9,excellent stayed hotel monaco past w/e delight...,4,35,"[[353, 497, 3, 236, 681, 267, 682, 683, 11, 53..."


In [82]:
# Revisamos las clases basados en el valor del rating
Counter(parsed_data['Rating'])

# las clases se encuentran medianamente desbalanceada en especial la clase 4, una posible idea que puede mejorar
# el la precision puede ser combinar la clase 0 1 2 en este caso especifico

Counter({3: 6039, 1: 1793, 2: 2184, 4: 9054, 0: 1421})

### 5. Definicion de la red recurrente LSTM



In [100]:
#Se crea la clase que se va a encargar de la magia 
class LSTM(torch.nn.Module) :
    """
    Inicialización de la clase.
    Parámetros:
        embedding_dim: dimesionalidad del vector de palabras. 
        hidden_dim: dimensión de la capa oculta de la red. 
        vocab_size: tamaño del vocabulario.  
        tagset_size: número de clases.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size=5) :
        super().__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, tagset_size)

        self.dropout = nn.Dropout(0.2)
    
    
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [93]:
def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        #loss = nn.CrossEntropyLoss(y_hat,y)
        #loss = torch.nn.en.cross_entropy(y_hat, y)
        loss = torch.nn.functional.cross_entropy(y_hat,y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

# Funcion se encarga de por cada epoch entrenar el modelo
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    # se intenta reducir el margen de error
    # con el algoritmo de Adam que ha demostrado ser muy efectivo en NLP
    optimizer = torch.optim.Adam(parameters, lr=lr)
    
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = torch.nn.functional.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, validation_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))


### Hiper-parametros

In [97]:
epochs = 10
batch_size = 5000
vocab_size = len(words)


### Separacion de las muestras y entrenamiento


In [86]:
# Creamos el conjunto de datos que se van a cargar  a la red 
class Dataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
    #Hacemos el override de los metodos de la clase padre
    def __len__(self):
        return len(self.y)
    #Hacemos el override de los metodos de la clase padre
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [87]:
# los arreglos con la data codificada y el rating
X = list(parsed_data['encoded'])
y = list(parsed_data['Rating'])

#70% de la data como datos de entrenamiento y 30% como datos de prueba
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

# Creación de los datasets de entrenamiento y validación
train_ds = Dataset(X_train, y_train)
valid_ds = Dataset(X_valid, y_valid)
# Creacion de los dataLoaders 
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
validation_dl = DataLoader(valid_ds, batch_size=batch_size)

In [98]:
# creamos el modelo

embebbed = 50
hidden = 50
learning_rate=0.01
clases = 5
model = LSTM(vocab_size=vocab_size,
            embedding_dim=embebbed,
            hidden_dim=hidden,
            tagset_size=5
)

train_model(model,
            epochs=epochs,
            lr=learning_rate
            )

train loss 1.404, val loss 1.357, val accuracy 0.439, and val rmse 1.594
train loss 1.324, val loss 1.360, val accuracy 0.435, and val rmse 1.563


In [99]:
# Validación del modelo 
average_loss, accuracy, average_rmse = validation_metrics (model, validation_dl)

print ("Exactitud", accuracy)
print("Raíz del error cuadrático medio", average_rmse)

Exactitud tensor(0.4253)
Raíz del error cuadrático medio 1.5163030836523577


### 8. Grafica de la curva de error

### 9. Matriz de confusion


### 10. Analisis de resultado

Podemos ver que la exactitud del tensor sin ningun cambio es alrededor del 52% 
y el error cuadratico medio 0.9333%

- Por lo tanto una mejora posible es unificar las clases puesto que no estan del todo balanceadas, unificar la clase 0 ,1 ,2 en una sola quedando en tres unicas clases (2,3,4)
- Aplicar stemming a las palabras puede mejorar el resultado ya que reduce el vocabulario por estudiar

### 11. Conclusiones
- Un buen manejo de las clases permite que sea mas exacto que el algoritmo no falle en dar el veredicto, es decir los datos de las clases deben de estar nivelados esto con el fin de que el algoritmo
de la respuesta correcta, y si da error este se denote como error.

- Manejar analisis de sentimiento asi como muchas otras redes recurrentes, no es el del todo facil, mucho de lo que se trabaja se maneja de manera heuristica, ya que no hay una formula exacta para modelar la realidad, es por esto que es necesaria la practica y el ensayo en estas pruebas.

- Un buen manejo del corpus puede mejorar el rendimiento asi como una buena normalizacion de los datos puede prevenir que el algoritmo tome informacion innesesaria, lo que puede provocar que el algoritmo se entrene de manera incorrecta por un lado asi como un gasto en rendimiento.

- El analisis de sentimiento nos permite de manera eficiente miles o millones de datos de manera rapida, hecho que a mano resulta muy lento o hasta imposible.
